In [1]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.02 MiB | 23.57 MiB/s, done.
Resolving deltas: 100% (10353/10353), done.


Under `darknet/Makefile` you’ll have to configure a couple of things. Change values on the following lines:

    Line 1 — From GPU=0 to GPU=1
    Line 2 — From CUDNN=0 to CUDNN=1
    Line 4 — From OPENCV=0 to OPENCV=1

In [2]:
%cd darknet
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [3]:
!cp cfg/yolov3.cfg cfg/yolov3-train.cfg

In [4]:
!ln -s /content/drive/MyDrive/ /mydrive

In [5]:
!echo -e 'license-plate\nvehicle' > data/obj.names 

In [6]:
!echo -e 'classes = 2\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolo-license-plates' > data/obj.data

In [7]:
!mkdir data/obj

In [8]:
!unzip ../data.zip -d data/obj

Archive:  ../data.zip
 extracting: data/obj/README.dataset.txt  
 extracting: data/obj/README.roboflow.txt  
   creating: data/obj/test/
 extracting: data/obj/test/_darknet.labels  
 extracting: data/obj/test/b15d6c0bdf90226d_jpg.rf.775557ba6aaf7d76a8d4bb04f66d8fc5.jpg  
 extracting: data/obj/test/b15d6c0bdf90226d_jpg.rf.775557ba6aaf7d76a8d4bb04f66d8fc5.txt  
 extracting: data/obj/test/b193070a9c45b5ab_jpg.rf.57e5987eb896a7bf9fc7a1a96a660c7e.jpg  
 extracting: data/obj/test/b193070a9c45b5ab_jpg.rf.57e5987eb896a7bf9fc7a1a96a660c7e.txt  
 extracting: data/obj/test/b1a50a3824887ee2_jpg.rf.68a4fd34fce20184287592f2680f895b.jpg  
 extracting: data/obj/test/b1a50a3824887ee2_jpg.rf.68a4fd34fce20184287592f2680f895b.txt  
 extracting: data/obj/test/b25d7d2cb7abf86b_jpg.rf.36edce738b0e69965e001035d6f85ab3.jpg  
 extracting: data/obj/test/b25d7d2cb7abf86b_jpg.rf.36edce738b0e69965e001035d6f85ab3.txt  
 extracting: data/obj/test/b32671b7cc279583_jpg.rf.f898026e2169a3b6adc4d21aee6d06c5.jpg  
 extract

In [14]:
import glob

train_images_list = glob.glob('data/obj/train/*.jpg')
test_images_list = glob.glob('data/obj/test/*.jpg')
with open('data/train.txt', 'w') as f:
  f.write('\n'.join(train_images_list))
with open('data/test.txt', 'w') as f:
  f.write('\n'.join(test_images_list))

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-04 18:47:11--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  22.8MB/s    in 7.4s    

2022-01-04 18:47:19 (20.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



And make the following changes in `cfg/yolov3-train.cfg`:

    Line 3 — From batch=1 to batch=64
    Line 4 — From subdivisions=1 to subdivisions=16
    Line 20 — From max_batches=500200 to max_batches=4000
    Lines 603, 689, and 776 — From filters=255 to filters=21
    Lines 610, 696, and 783 — From classes=80 to classes=2

In [10]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3-train.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-train.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3-train.cfg

!sed -i '610 s@classes=80@classes=2@' cfg/yolov3-train.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3-train.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3-train.cfg

!sed -i '603 s@filters=255@filters=21@' cfg/yolov3-train.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3-train.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3-train.cfg

In [ ]:
# !./darknet detector train data/obj.data cfg/yolov3-train.cfg darknet53.conv.74 -dont_show

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-train.cfg /mydrive/yolo-license-plates/yolov3-train_last.weights -dont_show

In [15]:
!./darknet detector map data/obj.data cfg/yolov3-train.cfg /mydrive/yolo-license-plates/yolov3-train_final.weights

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5, 